In [1]:
import sys
sys.path.append("..")

from tooling.agents.executor import Executor
from tooling.agents.skill import Skill
from tooling.agents.state import StateGraph
from tooling.agents.orchestrator.tools import select_function
from tooling.agents.orchestration_grader.tools import run_supervised_grading
from tooling.llm_engine.rayservice_wrapper import RayServiceWrapper as llm

In [2]:
from functools import partial


# create skills
class SkillSelector(Skill):
    # static skill definitions
    skill_name = "skill_selector"
    description: str = "select an available action"
    required_contexts: list[str] = []
    skill_metadata: dict = select_function.__annotations__
    # oss_function: ClassVar[dict] = None
    # oai_function: ClassVar[dict] = None

    prompt: str = ""
    contexts: dict = {}
    stream: bool = False
    use_oss: bool = False

    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

    def get_required_contexts(self):
        return self.required_contexts

    def get_prompt(self):
        return self.prompt

    async def build_prompt(self):
        pass

    def execute_skill(self, save_file_path:str="./state/orchestrator_selection.csv", **kwargs) -> dict[str,str]:
        df = select_function(**kwargs)
        df.to_csv(save_file_path, index=False, header=True)
        return {"orchestrator_csv": save_file_path}
    
class OrchestrationGrader(Skill):
    # static skill definitions
    skill_name = "orchestrator_grader"
    description: str = "grades orchestration results"
    required_contexts: list[str] = []
    skill_metadata: dict = run_supervised_grading.__annotations__
    # oss_function: ClassVar[dict] = None
    # oai_function: ClassVar[dict] = None

    prompt: str = ""
    contexts: dict = {}
    stream: bool = False
    use_oss: bool = False

    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

    def get_required_contexts(self):
        return self.required_contexts

    def get_prompt(self):
        return self.prompt

    async def build_prompt(self):
        pass

    def execute_skill(self, save_file_path:str="./state/grader.csv", **kwargs) -> dict[str,str]:
        df = run_supervised_grading(**kwargs)
        df.to_csv(save_file_path, index=False, header=True)
        return {"grader_csv": save_file_path}

In [4]:
from typing import Any

skill_selector = SkillSelector()
grader = OrchestrationGrader()

# Construct state graph
action_space = StateGraph()

# add actions
action_space.add_action(action_name=skill_selector.skill_name, skill=skill_selector)
action_space.add_action(action_name=grader.skill_name, skill=grader)

# define a condition
def state_contains_key(key:str, context:dict[str,Any]) -> bool:
    return key in context

# add conditional composition
composition_argument_map = {
    "orchestrator_csv": "data_file_path",
    "grading_prompt_id": "system_prompt_id"
}
action_space.add_composition(
    action_name_0=skill_selector.skill_name,
    action_name_1=grader.skill_name,
    argument_map=composition_argument_map,
    condition=partial(state_contains_key, key="orchestrator_csv")
)

print("actions")
for node in list(action_space.graph.nodes(data=True)):
    print(node)
print("\n")

print("compositions")
for edge in list(action_space.graph.edges(data=True)):
    print(edge)

actions
('skill_selector', {'skill_metadata': {'data_file_path': <class 'str'>, 'model': <class 'tooling.llm_engine.GenerativeModel'>, 'function_template_file_path': typing.Optional[str], 'prompting_strategy': typing.Optional[str], 'limit': typing.Optional[int], 'system_prompt_id': typing.Optional[str], 'ontology_prompt_id': typing.Optional[str], 'example_prompt_id': typing.Optional[str]}, 'invoke': functools.partial(<bound method SkillSelector.execute_skill of <__main__.SkillSelector object at 0x1754bba50>>)})
('orchestrator_grader', {'skill_metadata': {'data_file_path': <class 'str'>, 'model': <class 'tooling.llm_engine.GenerativeModel'>, 'reference_key': typing.Optional[str], 'candidate_key': typing.Optional[str], 'function_template_file_path': typing.Optional[str], 'limit': typing.Optional[int], 'system_prompt_id': typing.Optional[str], 'grading_prompt_id': typing.Optional[str], 'example_prompt_id': typing.Optional[str]}, 'invoke': functools.partial(<bound method OrchestrationGrade

In [5]:
from pathlib import Path
from argparse import ArgumentParser

def extract_arguments() -> ArgumentParser:
    parser = ArgumentParser()
    parser.add_argument("--input_file", type=str, default="../data/grading/orchestration/skill_test_new.csv")
    parser.add_argument("--output_directory", type=str, default="../data/grading/orchestration/agents")
    parser.add_argument("--function_template_file_path", type=str, default="../data/grading/orchestration/function_metadata.json")
    parser.add_argument("--limit", type=int, default=3)
    
    return parser.parse_args()

input_args = extract_arguments()
input_file=input_args.input_file
output_directory=input_args.output_directory
function_template_file_path=input_args.function_template_file_path
limit=input_args.limit

ORCHESTRATOR_OUTPUT_PATH = Path(output_directory, "selections.csv")
GRADER_OUTPUT_PATH = Path(output_directory, "grades.csv")

In [7]:
# instantiate executor
executor = Executor()
executor.add_state_graph(action_space)

# construct initial state if applicable
state = {
    "data_file_path":input_file,
    "function_template_file_path":"../data/grading/orchestration/function_metadata.json",
    "selector_prompt_id":"llama3_system",
    "grading_prompt_id":"llama3_system",
    "model":llm(),
    "limit":1
}

executor.augment_state(state)
executor.set_entry_point(action_name="skill_selector", argument_map={"selector_prompt_id": "system_prompt_id"})

# run executor
terminal_state = executor.run()

running skill_selector


  0%|          | 0/1 [00:00<?, ?it/s]

10it [00:02,  3.53it/s]                      
30it [00:01, 15.44it/s]                      
24it [00:02, 11.57it/s]                      
30it [00:07,  4.16it/s]                      
30it [00:03,  8.99it/s]                      
30it [00:02, 12.94it/s]                      
10it [00:13,  1.40s/it]                      


found new skills to run: ['orchestrator_grader']
running orchestrator_grader


164it [00:09, 16.79it/s]                     

found new skills to run: []
